# webLog_HW

- 과제 내용
    - 이전 실습 시간 때 다뤘던 weblog 데이터에서 조금 더 심화된 내용을 해보겠습니다.
    - 일전에 IP와 시간대를 기준으로 404 에러가 어떨 때 많이 뜰까? 란 문제의식을 갖고 실습을 했습니다.
    - 이번에는 "IP별로 로그인을 했을 때 얼마나 오랫동안 사이트를 이용할까?" 란 주제로 과제를 진행해보도록 하겠습니다.
    - 부연 설명을 하자면, 어떤 IP의 URL이 login.php를 포함하고 있다면 그 시점이 접속한 시간이고(inTime)
    - 다음 login.php가 나오기 직전(바로 위)의 row가 접속을 종료한 시간(outTime)이라고 가정하고
    - outTime에서 inTime을 빼서 duringTime을 계산하고
    - duringTime들의 평균을 계산하는 것이 과제의 목표입니다.
    
    
    
<hr>

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import collections

from pyspark import StorageLevel
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time
from pyspark.sql.functions import udf
from pyspark.sql.types import *

## 데이터 불러오고 dataFrame 생성

In [3]:
weblog = spark.read.csv("file:///home/ubuntu/18-2Engineering/Week05_181103/Spark_SQL_HW/webLog.csv")
weblog.show()

+----------+--------------------+--------------------+---+
|       _c0|                 _c1|                 _c2|_c3|
+----------+--------------------+--------------------+---+
|10.128.2.1|[29/Nov/2017:06:5...|GET /login.php HT...|200|
|10.128.2.1|[29/Nov/2017:06:5...|POST /process.php...|302|
|10.128.2.1|[29/Nov/2017:06:5...|GET /home.php HTT...|200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /js/vendor/mo...|200|
|10.130.2.1|[29/Nov/2017:06:5...|GET /bootstrap-3....|200|
|10.130.2.1|[29/Nov/2017:06:5...|GET /profile.php?...|200|
|10.128.2.1|[29/Nov/2017:06:5...|GET /js/jquery.mi...|200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /js/chart.min...|200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /edit.php?nam...|200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /logout.php H...|302|
|10.131.2.1|[29/Nov/2017:06:5...|GET /login.php HT...|200|
|10.130.2.1|[29/Nov/2017:07:0...|GET /login.php HT...|200|
|10.130.2.1|[29/Nov/2017:07:0...|GET /login.php HT...|200|
|10.130.2.1|[29/Nov/2017:13:3...|      GET / HTTP/1.1|30

## 컬럼명 생성

In [4]:
weblog = weblog.selectExpr("_c0 as IP", "_c1 as Time", "_c2 as URL", "_c3 as Status")
weblog.show()

+----------+--------------------+--------------------+------+
|        IP|                Time|                 URL|Status|
+----------+--------------------+--------------------+------+
|10.128.2.1|[29/Nov/2017:06:5...|GET /login.php HT...|   200|
|10.128.2.1|[29/Nov/2017:06:5...|POST /process.php...|   302|
|10.128.2.1|[29/Nov/2017:06:5...|GET /home.php HTT...|   200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /js/vendor/mo...|   200|
|10.130.2.1|[29/Nov/2017:06:5...|GET /bootstrap-3....|   200|
|10.130.2.1|[29/Nov/2017:06:5...|GET /profile.php?...|   200|
|10.128.2.1|[29/Nov/2017:06:5...|GET /js/jquery.mi...|   200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /js/chart.min...|   200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /edit.php?nam...|   200|
|10.131.2.1|[29/Nov/2017:06:5...|GET /logout.php H...|   302|
|10.131.2.1|[29/Nov/2017:06:5...|GET /login.php HT...|   200|
|10.130.2.1|[29/Nov/2017:07:0...|GET /login.php HT...|   200|
|10.130.2.1|[29/Nov/2017:07:0...|GET /login.php HT...|   200|
|10.130.

## IP, Time 기준으로 sorting 

In [5]:
weblog = weblog.sort(asc("IP"), asc("Time"))
weblog.show()

+----------+--------------------+--------------------+------+
|        IP|                Time|                 URL|Status|
+----------+--------------------+--------------------+------+
|10.128.2.1|[01/Dec/2017:00:4...|GET /login.php HT...|   200|
|10.128.2.1|[01/Dec/2017:04:1...|GET /bootstrap-3....|   200|
|10.128.2.1|[01/Dec/2017:04:1...|GET /css/style.cs...|   200|
|10.128.2.1|[01/Dec/2017:04:1...|GET /fonts/fontaw...|   200|
|10.128.2.1|[01/Dec/2017:04:1...|GET /sign.php HTT...|   200|
|10.128.2.1|[01/Dec/2017:04:1...|GET /description....|   200|
|10.128.2.1|[01/Dec/2017:05:5...|GET /js/vendor/mo...|   200|
|10.128.2.1|[01/Dec/2017:05:5...|GET /js/vendor/jq...|   200|
|10.128.2.1|[01/Dec/2017:05:5...|GET /css/style.cs...|   200|
|10.128.2.1|[01/Dec/2017:06:2...|GET /login.php HT...|   200|
|10.128.2.1|[01/Dec/2017:06:2...|GET /home.php HTT...|   200|
|10.128.2.1|[01/Dec/2017:06:2...|POST /process.php...|   302|
|10.128.2.1|[01/Dec/2017:06:2...|GET /contest.php ...|   200|
|10.128.

## Time에서 month 부분을 숫자로 변경, '['제거

In [6]:
# month는 'Jan', 'Feb', 'Mar', 'Nov', 'Dec'만 있습니다.
weblog = weblog.withColumn('T1', regexp_replace('Time', 'Jan', '01'))
weblog = weblog.withColumn('T2', regexp_replace("T1", 'Feb', '02'))
weblog = weblog.withColumn('T3', regexp_replace("T2", 'Mar', '03'))
weblog = weblog.withColumn('T4', regexp_replace("T3", 'Nov', '11'))
weblog = weblog.withColumn('Time_new', regexp_replace("T4", 'Dec', '12'))



def modify_time(Time_new):
     
    return substring(Time_new,2,21)

time = udf(modify_time, StringType())
weblog = weblog.withColumn("Time_new", modify_time("Time_new"))
weblog = weblog.drop(weblog['Time'])
weblog = weblog.drop(weblog['T1'])
weblog = weblog.drop(weblog['T2'])
weblog = weblog.drop(weblog['T3'])
weblog = weblog.drop(weblog['T4'])

weblog.show(2000)


+----------+--------------------+------+-------------------+
|        IP|                 URL|Status|           Time_new|
+----------+--------------------+------+-------------------+
|10.128.2.1|GET /login.php HT...|   200|01/12/2017:00:40:11|
|10.128.2.1|GET /css/style.cs...|   200|01/12/2017:04:10:01|
|10.128.2.1|GET /bootstrap-3....|   200|01/12/2017:04:10:01|
|10.128.2.1|GET /fonts/fontaw...|   200|01/12/2017:04:10:04|
|10.128.2.1|GET /sign.php HTT...|   200|01/12/2017:04:10:13|
|10.128.2.1|GET /description....|   200|01/12/2017:04:12:25|
|10.128.2.1|GET /js/vendor/jq...|   200|01/12/2017:05:52:08|
|10.128.2.1|GET /css/style.cs...|   200|01/12/2017:05:52:08|
|10.128.2.1|GET /js/vendor/mo...|   200|01/12/2017:05:52:08|
|10.128.2.1|GET /login.php HT...|   200|01/12/2017:06:23:58|
|10.128.2.1|GET /home.php HTT...|   200|01/12/2017:06:24:02|
|10.128.2.1|POST /process.php...|   302|01/12/2017:06:24:02|
|10.128.2.1|GET /contest.php ...|   200|01/12/2017:06:24:06|
|10.128.2.1|GET /archive

## Time format 변경 (01/12/2017:00:40:11 -> 20171201004011을 초 단위로 계산)

In [7]:
def year(Time_new):   
    return substring(Time_new,7,4)


def month(Time_new):
    return substring(Time_new,4,2)

def day(Time_new):
    return substring(Time_new,1,2)

def hour(Time_new):
    return substring(Time_new, 12,2)

def minute(Time_new):
    return substring(Time_new, 15,2)

def second(Time_new):
    return substring(Time_new, 18,2)



weblog = weblog.withColumn("Year", year("Time_new"))
weblog = weblog.withColumn("Month", month("Time_new"))
weblog = weblog.withColumn("Day", day("Time_new"))
weblog = weblog.withColumn("Hour", hour("Time_new"))
weblog = weblog.withColumn("Minute", minute("Time_new"))
weblog = weblog.withColumn("Second", second("Time_new"))

weblog = weblog.withColumn("Time", weblog.Second 
                           + weblog.Minute* 60 
                          + weblog.Hour*60*60
                          + weblog.Day * 60*60*24
                          + weblog.Month * 60*60*24*31
                          + weblog.Year * 60*60*24*31*12)



# weblog = weblog.withColumn("Time", concat(weblog.Year, weblog.Month, weblog.Day, weblog.Hour, weblog.Minute, weblog.Second))
weblog.show()


+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+
|10.128.2.1|GET /login.php HT...|   200|01/12/2017:00:40:11|2017|   12| 01|  00|    40|    11|6.4860223211E10|
|10.128.2.1|GET /bootstrap-3....|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|
|10.128.2.1|GET /css/style.cs...|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|
|10.128.2.1|GET /fonts/fontaw...|   200|01/12/2017:04:10:04|2017|   12| 01|  04|    10|    04|6.4860235804E10|
|10.128.2.1|GET /sign.php HTT...|   200|01/12/2017:04:10:13|2017|   12| 01|  04|    10|    13|6.4860235813E10|
|10.128.2.1|GET /description....|   200|01/12/2017:04:12:25|2017|   12| 01|  04|    12|    25|6.4860235945E10|
|

## IsLogin column 생성

In [8]:
col = functions.when(weblog["URL"].like('%login.php%'), True).otherwise(False).alias("IsLogin")
weblog = weblog.withColumn("IsLogin", col)
weblog.select("*").show()

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.128.2.1|GET /login.php HT...|   200|01/12/2017:00:40:11|2017|   12| 01|  00|    40|    11|6.4860223211E10|   true|
|10.128.2.1|GET /bootstrap-3....|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|  false|
|10.128.2.1|GET /css/style.cs...|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|  false|
|10.128.2.1|GET /fonts/fontaw...|   200|01/12/2017:04:10:04|2017|   12| 01|  04|    10|    04|6.4860235804E10|  false|
|10.128.2.1|GET /sign.php HTT...|   200|01/12/2017:04:10:13|2017|   12| 01|  04|    10|    13|6.4860235813E10|  false|
|10.128.2.1|GET /description....|   200|01/12/20

## IP 종류 확인

In [9]:
weblog.groupBy("IP").count().show()

+----------+-----+
|        IP|count|
+----------+-----+
|10.131.2.1| 1626|
|10.128.2.1| 2416|
|10.130.2.1| 2493|
|10.131.0.1| 2600|
|10.129.2.1| 1652|
+----------+-----+



## 1. IP 10.131.2.1

In [10]:
weblog1 = weblog.where(weblog["IP"]=="10.131.2.1")
weblog1.show()

inTime=0
outTime=0
sum=0
count=-1
duringTime=0

for row in weblog1.rdd.collect():    
    if row.IsLogin == True:
        sum += duringTime
        count +=1
        inTime = row.Time
    else:
        outTime = row.Time
        duringTime = int(outTime) - int(inTime)

        
        
average = sum / count
sec = average % 60
min = (average//60) % 60
hour = ((average//60) // 60) % 24
day = (((average//60)//60)//24)%31
print(day, "day", hour, "hour", min, "min", sec, 'sec')

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.131.2.1|GET /login.php HT...|   200|01/12/2017:00:06:02|2017|   12| 01|  00|    06|    02|6.4860221162E10|   true|
|10.131.2.1|      GET / HTTP/1.1|   302|01/12/2017:04:09:59|2017|   12| 01|  04|    09|    59|6.4860235799E10|  false|
|10.131.2.1|GET /css/font-awe...|   200|01/12/2017:04:10:00|2017|   12| 01|  04|    10|    00|  6.48602358E10|  false|
|10.131.2.1|GET /contest.php ...|   200|01/12/2017:04:11:46|2017|   12| 01|  04|    11|    46|6.4860235906E10|  false|
|10.131.2.1|GET /archive.php ...|   200|01/12/2017:04:12:09|2017|   12| 01|  04|    12|    09|6.4860235929E10|  false|
|10.131.2.1|GET /login.php HT...|   200|01/12/20

## 2. IP 10.128.2.1

In [11]:
weblog2 = weblog.where(weblog["IP"]=="10.128.2.1")
weblog2.show()

inTime=0
outTime=0
sum=0
count=-1
duringTime=0

for row in weblog2.rdd.collect():    
    if row.IsLogin == True:
        sum += duringTime
        count +=1
        inTime = row.Time
    else:
        outTime = row.Time
        duringTime = int(outTime) - int(inTime)

        
        
average = sum / count
sec = average % 60
min = (average//60) % 60
hour = ((average//60) // 60) % 24
day = (((average//60)//60)//24)%31
print(day, "day", hour, "hour", min, "min", sec, 'sec')

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.128.2.1|GET /login.php HT...|   200|01/12/2017:00:40:11|2017|   12| 01|  00|    40|    11|6.4860223211E10|   true|
|10.128.2.1|GET /bootstrap-3....|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|  false|
|10.128.2.1|GET /css/style.cs...|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|  false|
|10.128.2.1|GET /fonts/fontaw...|   200|01/12/2017:04:10:04|2017|   12| 01|  04|    10|    04|6.4860235804E10|  false|
|10.128.2.1|GET /sign.php HTT...|   200|01/12/2017:04:10:13|2017|   12| 01|  04|    10|    13|6.4860235813E10|  false|
|10.128.2.1|GET /description....|   200|01/12/20

## 3. IP 10.130.2.1

In [12]:
weblog3 = weblog.where(weblog["IP"]=="10.130.2.1")
weblog3.show()

inTime=0
outTime=0
sum=0
count=-1
duringTime=0

for row in weblog3.rdd.collect():    
    if row.IsLogin == True:
        sum += duringTime
        count +=1
        inTime = row.Time
    else:
        outTime = row.Time
        duringTime = int(outTime) - int(inTime)

        
        
average = sum / count
sec = average % 60
min = (average//60) % 60
hour = ((average//60) // 60) % 24
day = (((average//60)//60)//24)%31
print(day, "day", hour, "hour", min, "min", sec, 'sec')

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.130.2.1|      GET / HTTP/1.1|   302|01/12/2017:00:40:11|2017|   12| 01|  00|    40|    11|6.4860223211E10|  false|
|10.130.2.1|      GET / HTTP/1.1|   302|01/12/2017:00:45:17|2017|   12| 01|  00|    45|    17|6.4860223517E10|  false|
|10.130.2.1|GET /login.php HT...|   200|01/12/2017:00:45:18|2017|   12| 01|  00|    45|    18|6.4860223518E10|   true|
|10.130.2.1|GET /img/ruet.png...|   200|01/12/2017:01:30:30|2017|   12| 01|  01|    30|    30| 6.486022623E10|  false|
|10.130.2.1|GET /css/normaliz...|   200|01/12/2017:04:10:00|2017|   12| 01|  04|    10|    00|  6.48602358E10|  false|
|10.130.2.1|GET /js/vendor/mo...|   200|01/12/20

## 4. IP 10.131.0.1

In [13]:
weblog4 = weblog.where(weblog["IP"]=="10.131.0.1")
weblog4.show()

inTime=0
outTime=0
sum=0
count=-1
duringTime=0

for row in weblog4.rdd.collect():    
    if row.IsLogin == True:
        sum += duringTime
        count +=1
        inTime = row.Time
    else:
        outTime = row.Time
        duringTime = int(outTime) - int(inTime)

        
        
average = sum / count
sec = average % 60
min = (average//60) % 60
hour = ((average//60) // 60) % 24
day = (((average//60)//60)//24)%31
print(day, "day", hour, "hour", min, "min", sec, 'sec')

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.131.0.1|POST /process.php...|   302|01/12/2017:00:41:00|2017|   12| 01|  00|    41|    00| 6.486022326E10|  false|
|10.131.0.1|GET /home.php HTT...|   200|01/12/2017:00:41:00|2017|   12| 01|  00|    41|    00| 6.486022326E10|  false|
|10.131.0.1|GET /contestprobl...|   200|01/12/2017:00:41:05|2017|   12| 01|  00|    41|    05|6.4860223265E10|  false|
|10.131.0.1|GET /details.php?...|   200|01/12/2017:00:41:28|2017|   12| 01|  00|    41|    28|6.4860223288E10|  false|
|10.131.0.1|GET /login.php HT...|   200|01/12/2017:04:10:00|2017|   12| 01|  04|    10|    00|  6.48602358E10|   true|
|10.131.0.1|GET /css/bootstra...|   200|01/12/20

## 5. IP 10.129.2.1

In [14]:
weblog5 = weblog.where(weblog["IP"]=="10.129.2.1")
weblog5.show()

inTime=0
outTime=0
sum=0
count=-1
duringTime=0

for row in weblog5.rdd.collect():    
    if row.IsLogin == True:
        sum += duringTime
        count +=1
        inTime = row.Time
    else:
        outTime = row.Time
        duringTime = int(outTime) - int(inTime)

        
        
average = sum / count
sec = average % 60
min = (average//60) % 60
hour = ((average//60) // 60) % 24
day = (((average//60)//60)//24)%31
print(day, "day", hour, "hour", min, "min", sec, 'sec')

+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|        IP|                 URL|Status|           Time_new|Year|Month|Day|Hour|Minute|Second|           Time|IsLogin|
+----------+--------------------+------+-------------------+----+-----+---+----+------+------+---------------+-------+
|10.129.2.1|      GET / HTTP/1.1|   302|01/12/2017:00:06:02|2017|   12| 01|  00|    06|    02|6.4860221162E10|  false|
|10.129.2.1|GET /css/main.css...|   200|01/12/2017:04:10:00|2017|   12| 01|  04|    10|    00|  6.48602358E10|  false|
|10.129.2.1|GET /js/vendor/jq...|   200|01/12/2017:04:10:01|2017|   12| 01|  04|    10|    01|6.4860235801E10|  false|
|10.129.2.1|GET /css/normaliz...|   200|01/12/2017:05:52:08|2017|   12| 01|  05|    52|    08|6.4860241928E10|  false|
|10.129.2.1|GET /standings.ph...|   302|01/12/2017:06:11:34|2017|   12| 01|  06|    11|    34|6.4860243094E10|  false|
|10.129.2.1|GET /contestsubmi...|   302|01/12/20